## INSTALLING THE NECESSARY DEPENDENCIES:

In [ ]:
##(install and restart runtime)
#!pip install --upgrade accelerate
#!pip install transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 78.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.2 MB/s eta 0:00:00


In [ ]:
import torch
import numpy as np
import pandas as pd

!pip install transformers datasets huggingface_hub tensorboard==2.11
!sudo apt-get install git-lfs --yes
import os

from datasets import Dataset
from datasets import load_metric
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import TrainingArguments, AutoModelForSequenceClassification, Trainer

torch.__version__


from huggingface_hub import HfFolder, notebook_login

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 82.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 113.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.1
    Uninstalling tensorboard-data-server-0.7.1:
      Successfully uninstalled tensorboard-data-server-0.7.1
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 1.0.0
    Uninstalling google-auth-oauthlib-1.0.0:
      Successfully uninstalled google-auth-oauthlib-1.0.0
  Attemp

In [ ]:
notebook_login()

In [ ]:
repository_id = "Geee05/nltk_5000_electra"

In [ ]:
dataset = pd.read_csv('/content/All_transcript_train.csv')
tdataset= pd.read_csv('/content/All_transcript_test_new.csv')

## DATA PRE-PROCESSING

In [ ]:
dataset.rename(columns = {'Class':'labels'}, inplace = True)
tdataset.rename(columns = {'Group':'labels'}, inplace = True)

In [ ]:
dataset.replace({'labels':{"HC":0,"DP":1}},inplace=True)
tdataset.replace({'labels':{"HC":0,"DP":1}},inplace=True)

In [ ]:
dataset['labels'].value_counts()

0    126
1     37
Name: labels, dtype: int64

In [ ]:
#resample data to deal with class imbalance
from sklearn.utils import resample
df_1=dataset[dataset['labels']==1]
df_2=dataset[dataset['labels']!=1]
df_upsampled=resample(df_1,random_state=42,n_samples=126,replace=True)
df=pd.concat([df_upsampled,df_2])
df.reset_index(inplace=True)

In [ ]:
import re
from gensim.parsing.preprocessing import remove_stopwords
import string
import unicodedata

def lower_case(text):
    return text.lower()

def remove_punctuation(text):
    text_list = [char for char in text if char not in string.punctuation]
    clean_text = "".join(text_list)
    return clean_text

def remove_numbers(text):
    text_list = []
    for i in text:
        if not re.search("\d", i):
            text_list.append(i)
    return "".join(text_list)

def remove_special_char(text):
    pattern =  r'[^a-zA-z0-9]'
    return re.sub(pattern, " ", text)


def remove_square_bracket(text):
    clean_text = re.sub(r'\[[0-9]*\]', ' ', text)
    return clean_text

def remove_white_spaces_char(text):
    clean_text = re.sub(r'\s+', ' ', text)

    return clean_text

def remove_unicode_char(text):
    new_text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return new_text


def remove_user(text):
    clean_text = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', text)

    return clean_text

def remove_links(text):
    clean_text = re.sub(r'http\S+', '', text)
    clean_text = re.sub(r'bit.ly/\S+', '', text)
    clean_text = clean_text.strip('[link]')

    return clean_text

def remove_html(text):
    html = re.compile('<.*?>')

    return html.sub(r'',text)

def remove_email(text):
    email = re.compile(r'[\w\.-]+@[\w\.-]+')
    return email.sub(r'',text)

def remove_stopwords_from_text(text):
    clean_text = remove_stopwords(text)
    return clean_text

In [ ]:
df["clean_text"] = df["Text"].apply(lower_case)
df["clean_text"] = df["clean_text"].apply(remove_punctuation)
df["clean_text"] = df["clean_text"].apply(remove_numbers)
df["clean_text"] = df["clean_text"].apply(remove_special_char)
df["clean_text"] = df["clean_text"].apply(remove_square_bracket)
df["clean_text"] = df["clean_text"].apply(remove_white_spaces_char)
df["clean_text"] = df["clean_text"].apply(remove_unicode_char)
df["clean_text"] = df["clean_text"].apply(remove_user)
df["clean_text"] = df["clean_text"].apply(remove_links)
df["clean_text"] = df["clean_text"].apply(remove_html)
df["clean_text"] = df["clean_text"].apply(remove_email)
df["clean_text"] = df["clean_text"].apply(remove_stopwords_from_text)

In [ ]:
tdataset["clean_text"] = tdataset["Text"].apply(lower_case)
tdataset["clean_text"] = tdataset["clean_text"].apply(remove_punctuation)
tdataset["clean_text"] = tdataset["clean_text"].apply(remove_numbers)
tdataset["clean_text"] = tdataset["clean_text"].apply(remove_special_char)
tdataset["clean_text"] = tdataset["clean_text"].apply(remove_square_bracket)
tdataset["clean_text"] = tdataset["clean_text"].apply(remove_white_spaces_char)
tdataset["clean_text"] = tdataset["clean_text"].apply(remove_unicode_char)
tdataset["clean_text"] = tdataset["clean_text"].apply(remove_user)
tdataset["clean_text"] = tdataset["clean_text"].apply(remove_links)
tdataset["clean_text"] = tdataset["clean_text"].apply(remove_html)
tdataset["clean_text"] = tdataset["clean_text"].apply(remove_email)
tdataset["clean_text"] = tdataset["clean_text"].apply(remove_stopwords_from_text)

In [ ]:
#creating Datasets
train = Dataset.from_pandas(df, preserve_index=False)
test = Dataset.from_pandas(tdataset, preserve_index=False)

print(train)
print(test)

Dataset({
    features: ['index', 'Unnamed: 0', 'File', 'Text', 'labels', 'clean_text'],
    num_rows: 252
})
Dataset({
    features: ['Unnamed: 0.1', 'Unnamed: 0', 'File', 'Text', 'labels', 'clean_text'],
    num_rows: 56
})


In [ ]:
leng=[]
for i in range (0,len(train['clean_text'])):
  leng.append(len(train['clean_text'][i]))

print(min(leng))
print(max(leng))

734
10609


## FUNCTION DEFINITION

In [ ]:
#SEGMENT TEXTS
def segment_function(example):
  seg=example['clean_text']
  res=[]
  length=[]
  segmented_file=[]
  n=5000

  for iter in range (0,len(seg)):
    count=0
    for i in range (0,len(seg[iter])):
      j=i*2500
      if len(seg[iter])<5000:
          part=(seg[iter])
          res.append(part)
          count=1
          break
      elif j<=len(seg[iter]):
          segmented_file.append((example['File'][iter]))
          part=(seg[iter][j:j+n])
          res.append(part)
          #print(iter)

          count=count+1
      else:
        break
    length.append(count)

  return res#,segmented_file,length

#TOKENIZE TEXTS
tokenizer = AutoTokenizer.from_pretrained('gooohjy/suicidal-electra')
def tokenize_function(example):
  return tokenizer(example["Text"], padding='max_length',max_length=512,truncation=True,add_special_tokens = True)

In [ ]:
#LABELS CORR. TO SEGMENTS
label=[]
for i in range (0,len(train)):
  if (len(train['clean_text'][i])) <5000:
    label.append(train['labels'][i])
  else:
    x = ((len(train['clean_text'][i]))//2500) + 1
    for j in range (0,x):
      label.append(train['labels'][i])



label_t=[]
for i in range (0,len(test)):
  if (len(test['clean_text'][i])) <5000:
    label_t.append(test['labels'][i])
  else:
    x = ((len(test['clean_text'][i]))//2500) + 1
    for j in range (0,x):
      label_t.append(test['labels'][i])

#### Segmenting Train Data

In [ ]:
#ip_seg,index,leng=segment_function(train)
ip_seg=segment_function(train)

In [ ]:
len(train['clean_text'][5])

3323

In [ ]:
len(ip_seg[7])

3323

In [ ]:
print(ip_seg[5])
print('------------')
print(ip_seg[6])

ows logically wasnt fault able deal chemicals brain probably different probably gone differently maybe wouldnt mean dont know dont know different things mean possible mean obviously yeah mean thats little mean little things think time head drives crazy dwell stuff like little comes suddenly remember throw day mean uncomfortable embarrassed feel ashamed stuff high school years years ago shouldnt thinking anymore cuz past feeling like im right sounds hard easy ive trying figure ways deal mean dealt long time drinking know trying slow mind way know medication better impossible deal easy better like said lot know im im writing im like kind things occupy brain completely im watching tv playing game phone im know watching tv movie know window mind occupied need mean wish sort let thoughts come deal need triggered theyre triggered pretty said hear somebody dutch hear somebody talk wisconsin hear know talk anyting know reach connect dont know thats thing mean dont break easy im light sleeper i

In [ ]:
#preparing dataframe for segmented texts & corresponding labels
train_data=pd.DataFrame(ip_seg,columns=['Text'])
train_data['labels']=label
train_data.head(2)

,Text,labels
0,okay hi im ellie thanks coming today created t...,1
1,okay yes im alright pretty good little bit tir...,1


#### Tokenizing Train Data

In [ ]:
trainset = Dataset.from_pandas(train_data, preserve_index=False)
trainset

Dataset({
    features: ['Text', 'labels'],
    num_rows: 356
})

In [ ]:
tokenized_train_dataset = trainset.map(tokenize_function, batched=True)
tokenized_train_dataset

Map:   0%|          | 0/356 [00:00<?, ? examples/s]

Dataset({
    features: ['Text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 356
})

#### Segmenting Test Data

In [ ]:
ip_seg_t=segment_function(test)

test_data=pd.DataFrame(ip_seg_t,columns=['Text'])
test_data['labels']=label_t
test_data.head(2)

,Text,labels
0,okay coming ahead shrink continue setting some...,0
1,super need like home wow okay alrighty yes hop...,1


#### Tokenizing Test Data

In [ ]:
testset = Dataset.from_pandas(test_data, preserve_index=False)
testset

Dataset({
    features: ['Text', 'labels'],
    num_rows: 71
})

In [ ]:
tokenized_test_dataset = testset.map(tokenize_function, batched=True)
tokenized_test_dataset

Map:   0%|          | 0/71 [00:00<?, ? examples/s]

Dataset({
    features: ['Text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 71
})

## REVIEW:

In [ ]:
print(len(label))
print(len(label_t))


356
71


In [ ]:
print(len(ip_seg))
print(len(ip_seg_t))


356
71


In [ ]:
type(tokenized_train_dataset)

datasets.arrow_dataset.Dataset

## TRAINING

In [ ]:
# Set dataset format using set_format() function to specify the dataset format, making it compatible with PyTorch
tokenized_train_dataset.set_format("torch", columns=['input_ids','attention_mask','labels'])
tokenized_test_dataset.set_format("torch", columns=['input_ids','attention_mask','labels'])

In [ ]:
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from sklearn.metrics import classification_report

training_args = TrainingArguments(evaluation_strategy="epoch",num_train_epochs=35,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_steps=500,
    output_dir=repository_id,
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    metric_for_best_model="f1",
    greater_is_better=True,
    report_to="tensorboard",
    push_to_hub=True,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token())   # default arguments for fine-tuning
model = AutoModelForSequenceClassification.from_pretrained('gooohjy/suicidal-electra', num_labels=2) #,ignore_mismatched_sizes=True
def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    #probabilities = softmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}
    print((classification_report(labels,predictions)))

trainer = Trainer(   # specifying trainer class
    model,
    training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    #data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


Cloning https://huggingface.co/Geee05/nltk_5000_electra into local empty directory.


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.083400,1.759446,0.521127,0.484848
2,0.561200,1.165982,0.591549,0.553846
3,0.513500,0.661073,0.690141,0.450000
4,0.616300,1.052297,0.647887,0.545455
5,0.477300,1.161883,0.605634,0.562500
6,0.252100,1.578826,0.704225,0.533333
7,0.304000,1.957954,0.619718,0.526316
8,0.145800,1.912337,0.718310,0.500000
9,0.126700,2.053338,0.718310,0.444444
10,0.059500,2.878397,0.633803,0.566667


<ipython-input-28-a51742d4a3cf>:25: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


TrainOutput(global_step=1575, training_loss=0.16295489028656102, metrics={'train_runtime': 2539.6048, 'train_samples_per_second': 4.906, 'train_steps_per_second': 0.62, 'total_flos': 3278363749785600.0, 'train_loss': 0.16295489028656102, 'epoch': 35.0})

In [ ]:
# Save our tokenizer and create a model card
tokenizer.save_pretrained(repository_id)
trainer.create_model_card()
# Push the results to the hub
trainer.push_to_hub()

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 1.00/418M [00:00<?, ?B/s]

Upload file logs/events.out.tfevents.1689758549.908362b1e3f0.1416.0:   0%|          | 1.00/41.1k [00:00<?, ?B/…

To https://huggingface.co/Geee05/nltk_5000_electra
   5bd663a..a04cd1f  main -> main

   5bd663a..a04cd1f  main -> main



'https://huggingface.co/Geee05/nltk_5000_electra/commit/a04cd1f5230db23d30fc1a2550093d4bff530b2c'

## VALIDATION

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("Geee05/overlap3")
model_electra = AutoModelForSequenceClassification.from_pretrained("Geee05/overlap3")

In [ ]:
vdataset= pd.read_csv('/content/All_transcript_val_new.csv')

In [ ]:
vdataset.head(2)

,Unnamed: 0.1,Unnamed: 0,File,Text,Group
0,0,0,600_AUDIO,okay there she is coming to go ahead and Shrin...,HC
1,1,1,602_AUDIO,this is super need I like this me either at a...,DP


In [ ]:
vdataset.rename(columns = {'Group':'labels'}, inplace = True)
vdataset.replace({'labels':{"HC":0,"DP":1}},inplace=True)
vdataset['labels'].value_counts()

0    39
1    17
Name: labels, dtype: int64

In [ ]:
vdataset["clean_text"] = vdataset["Text"].apply(lower_case)
vdataset["clean_text"] = vdataset["clean_text"].apply(remove_punctuation)
vdataset["clean_text"] = vdataset["clean_text"].apply(remove_numbers)
vdataset["clean_text"] = vdataset["clean_text"].apply(remove_special_char)
vdataset["clean_text"] = vdataset["clean_text"].apply(remove_square_bracket)
vdataset["clean_text"] = vdataset["clean_text"].apply(remove_white_spaces_char)
vdataset["clean_text"] = vdataset["clean_text"].apply(remove_unicode_char)
vdataset["clean_text"] = vdataset["clean_text"].apply(remove_user)
vdataset["clean_text"] = vdataset["clean_text"].apply(remove_links)
vdataset["clean_text"] = vdataset["clean_text"].apply(remove_html)
vdataset["clean_text"] = vdataset["clean_text"].apply(remove_email)
vdataset["clean_text"] = vdataset["clean_text"].apply(remove_stopwords_from_text)

In [ ]:
val = Dataset.from_pandas(vdataset, preserve_index=False)
print(val)

Dataset({
    features: ['Unnamed: 0.1', 'Unnamed: 0', 'File', 'Text', 'labels', 'clean_text'],
    num_rows: 56
})


In [ ]:
#LABELS CORR. TO SEGMENTS

label_v=[]
for i in range (0,len(val)):
  if (len(val['clean_text'][i])) <5000:
    label_v.append(val['labels'][i])
  else:
    x = ((len(val['clean_text'][i]))//2500) + 1
    for j in range (0,x):
      label_v.append(val['labels'][i])

In [ ]:
ip_seg_v=segment_function(val)

In [ ]:
print(len(label_v))
print(len(ip_seg_v))

71
71


In [ ]:
ip_seg_v=segment_function(val)

val_data=pd.DataFrame(ip_seg_v,columns=['Text'])
val_data['labels']=label_v
val_data.head(2)

,Text,labels
0,okay coming ahead shrink continue setting some...,0
1,super need like home wow okay alrighty yes hop...,1


In [ ]:
valset = Dataset.from_pandas(val_data, preserve_index=False)
valset

Dataset({
    features: ['Text', 'labels'],
    num_rows: 71
})

In [ ]:
tokenized_val_dataset = valset.map(tokenize_function, batched=True)
tokenized_val_dataset

Map:   0%|          | 0/71 [00:00<?, ? examples/s]

Dataset({
    features: ['Text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 71
})

In [ ]:
tokenized_val_dataset.set_format("torch", columns=['input_ids','attention_mask','labels'])

In [ ]:
tokenized_val_dataset

Dataset({
    features: ['Text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 71
})

In [ ]:
tokenized_val_dataset['input_ids'].shape

torch.Size([71, 512])

In [ ]:
logits=[]
labels=[]
for i in range (0,len(tokenized_val_dataset['Text'])):

  inputs = tokenizer(tokenized_val_dataset['Text'][i],padding='max_length',max_length=512,truncation=True,add_special_tokens = True, return_tensors="pt")
  with torch.no_grad():
    outputs = model_electra(**inputs)
    logits_l = model_electra(**inputs).logits
    #print(logits)
    logits.append(logits_l)
    labels_i = logits_l.argmax().item()
  labels.append(labels_i)

  #print(labels)

    #print(outputs)
  #print(np.array(inputs['input_ids']).shape)
  #print(np.array(inputs['token_type_ids']).shape)
  #print(np.array(inputs['attention_mask']).shape)
  #print(**inputs.shape)

  #last_hidden_states = outputs.last_hidden_state
  #result.append(outputs)

In [ ]:
len(labels)

71

In [ ]:
len(logits)

71

In [ ]:
true_labels=[]
for i in range (0,len(tokenized_val_dataset['labels'])):
  true_labels.append(tokenized_val_dataset['labels'][i])


In [ ]:
len(true_labels)

71

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(true_labels,labels))

              precision    recall  f1-score   support

           0       0.71      0.96      0.82        49
           1       0.60      0.14      0.22        22

    accuracy                           0.70        71
   macro avg       0.66      0.55      0.52        71
weighted avg       0.68      0.70      0.63        71

